In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import keras
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [54]:
data = pd.read_csv('book32-listing.csv',encoding = "ISO-8859-1")

In [55]:
columns = ['Id', 'Image', 'Image_link', 'Title', 'Author', 'Class', 'Genre']
data.columns = columns

In [56]:
data = data.dropna()

In [57]:
#data['Author'] = data['Author'].fillna('No Book')
#data['Title'] = data['Title'].fillna('No Book')
#data['Genre'] = data['Genre'].fillna('No Book')

books = pd.DataFrame(data['Title'])
author = pd.DataFrame(data['Author'])
genre = pd.DataFrame(data['Genre'])

In [6]:
from sklearn.preprocessing import LabelEncoder

feat = ['Genre']
for x in feat:
    le = LabelEncoder()
    le.fit(list(genre[x].values))
    genre[x] = le.transform(list(genre[x]))
    

In [58]:
genre['Genre'][:10]

0     Calendars
1     Calendars
2     Calendars
3     Calendars
4     Calendars
5     Calendars
7     Calendars
8     Calendars
9     Calendars
10    Calendars
Name: Genre, dtype: object

In [8]:
data.shape

(193172, 7)

In [59]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

feat = ['Genre']
for x in feat:
    le = LabelEncoder()
    le.fit(list(data[x].values))
    data[x] = le.transform(list(data[x]))
    #le.fit(x)
    #encoded_Y = le.transform(x)
    # convert integers to dummy variables (i.e. one hot encoded)
    y = np_utils.to_categorical(data[x])

In [60]:
y.shape

(193172, 32)

In [61]:
y[0]

array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.])

In [38]:
data['everything'] = pd.DataFrame(data['Title'] + ' ' + data['Author'])
#data['everything'] = data['everything'].fillna('No Book')
data['everything'].head(5)

0         Doug the Pug 2016 Wall Calendar Doug the Pug
1    Moleskine 2016 Weekly Notebook, 12M, Large, Bl...
2    365 Cats Color Page-A-Day Calendar 2016 Workma...
3     Sierra Club Engagement Calendar 2016 Sierra Club
4     Sierra Club Wilderness Calendar 2016 Sierra Club
Name: everything, dtype: object

In [39]:
data['everything'].shape

(193172,)

In [ ]:
(data['everything'][193192])

In [ ]:
text = []
for x in data['everything'][:]:
    text.append(x)

In [ ]:
len(text)

In [ ]:
text[:10]

In [ ]:
all_text = '\n'.join([t for t in (text)])

In [ ]:
(all_text)[:1000]

In [ ]:
everything = all_text.split('\n')

all_text = ' '.join(everything)
words = all_text.split()

In [ ]:
words[:10]

In [40]:
everything = pd.DataFrame(data['everything'])

In [62]:
## Counting how many times a word appears in the dataset
from collections import Counter

total_counts = Counter()
for i in range(len(books)):
    for word in data['Title'].values[i].split(" "):
        total_counts[word] += 1

print("Total words in data set: ", len(total_counts))

Total words in data set:  143838


In [63]:
## Sorting in decreasing order (Word with highest frequency appears first)
## We only use the first 10000 words for our vocab
vocab = sorted(total_counts, key=total_counts.get, reverse=True)[:2000]
print(vocab[:60])

['and', 'of', 'The', 'the', 'to', 'A', 'for', 'in', 'Guide', '&', 'a', 'Your', 'with', 'How', 'Book', 'Edition', 'Edition)', 'on', 'from', '-', 'Series)', 'New', 'Life', 'An', 'You', 'History', 'World', 'Recipes', 'Complete', 'For', '(The', 'Story', 'American', 'My', 'by', 'Travel', 'Art', '(Volume', '', 'To', 'Law', 'And', 'From', 'Novel', 'at', 'Health', 'In', 'What', 'Best', 'Calendar', 'Volume', 'I', 'With', 'Handbook', 'Introduction', 'Practice', 'Great', 'Of', 'That', 'Business']


In [64]:
print (vocab[-1])
total_counts[vocab[-1]]

Parent's


106

In [65]:
# Mapping from words to index

vocab_size = len(vocab)
word2idx = {}
#print vocab_size
for i, word in enumerate(vocab):
    word2idx[word] = i

In [66]:
### Text to Vector
def text_to_vector(text):
    word_vector = np.zeros(vocab_size)
    for word in text.split(" "):
        if word2idx.get(word) is None:
            continue
        else:
            word_vector[word2idx.get(word)] += 1
    return np.array(word_vector)

In [68]:
## Convert all titles to vectors
word_vectors = np.zeros((len(data['Title']), len(vocab)), dtype=np.int_)
for ii, (_, text) in enumerate(books.iterrows()):
    word_vectors[ii] = text_to_vector(text[0])

In [69]:
word_vectors.shape

(193172, 2000)

## Main Implementation

In [ ]:
from collections import Counter
split_words = Counter(words)
sorted_split_words = sorted(split_words, key=split_words.get, reverse=True)
vocab_to_int = {c : i for i, c in enumerate(sorted_split_words,1)}

# Convert the reviews to integers, same shape as reviews list, but with integers
books = text
book_ints = []
for book in books:
    book_ints.append([vocab_to_int[i] for i in book.split()])

In [ ]:
print (len(book_ints))
book_ints[:10]

In [ ]:
seq_len = 50
features = np.zeros((len(book_ints), seq_len), dtype=int)
for i, row in enumerate(book_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [ ]:
features[10]

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [71]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data['Title'])
vectors.shape

(193172, 67105)

In [ ]:
#genre['Genre'].unique()

In [ ]:
#genre['Genre'] = keras.utils.to_categorical((genre['Genre']),num_classes = 32)

In [72]:
from sklearn.cross_validation import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(word_vectors[:50000], y[:50000], test_size=0.2)

In [73]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(40000, 2000)
(40000, 32)
(10000, 2000)
(10000, 32)


In [74]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

clf = MultinomialNB(alpha=.005)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

ValueError: bad input shape (40000, 32)

In [25]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(solver= 'sag',max_iter=500,random_state=450)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

KeyboardInterrupt: 

In [85]:
batch_size = 128
print('Build model...')
model = Sequential()
model.add(Embedding(4000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.4))
model.add(Dense(32, activation='softmax'))

# try using different optimizers and different optimizer configs


Build model...


In [86]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [87]:
print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))


Train...
Train on 40000 samples, validate on 10000 samples
Epoch 1/1
  384/40000 [..............................] - ETA: 7296s - loss: 3.4473 - acc: 0.1042

KeyboardInterrupt: 

In [84]:
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)

  224/10000 [..............................] - ETA: 531s

KeyboardInterrupt: 

In [ ]:
print('Test score:', score)
print('Test accuracy:', acc)